In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [3]:
test_set = pd.read_csv("../data/ml-1m/testing_set_seed123_ml1m.csv",sep=",",
    header=None,
    names=["userID", "itemID", "Rating"],)
test_set


,userID,itemID,Rating
0,6039,373,4
1,6039,221,3
2,6039,2602,4
3,6039,132,4
4,6039,350,4
...,...,...,...
202331,2946,885,3
202332,2946,343,3
202333,2946,151,4
202334,2946,1792,4


In [4]:
all_users_tp = test_set.groupby('userID')['itemID'].apply(list).tolist()
all_users_tp.__len__()

6040

In [11]:
wmf = "model_reco/1m/wmf/reco_matrix_wmf_1m_1000_.npy"
reco_matrix = np.load(f"../{wmf}")
reco_matrix.shape

(9, 6040, 20)

In [ ]:

from cornac.metrics import RMSE, AUC, NDCG, Precision, Recall,HitRatio

hr=HitRatio(20)
alist=[]
o=1
sum_hr=0
for u in range(reco_matrix[o].shape[0]):
    x=hr.compute(all_users_tp[u],reco_matrix[o][u][:20])
    sum_hr=sum_hr+x
plain_hr=sum_hr/reco_matrix[o].shape[0]

plain_hr


IndexError: index 9 is out of bounds for axis 0 with size 9

In [ ]:
0.4374172185430464
0.44850993377483445
0.43940397350993377
0.4183774834437086
0.4013245033112583
0.3870860927152318
0.3844370860927152
0.3751655629139073
0.3741721854304636

In [31]:
# from NDCG import NDCG
ndcg=NDCG(20)
alist=[]
sum_ndcg=0
o = 8
for u in range(reco_matrix[0].shape[0]):
    x=ndcg.compute(all_users_tp[u],reco_matrix[o][u][:20])
    
    # print(f"x {x} tp {all_users_tp[u]} reco {reco_matrix[u][:20]}")
    # alist.append(x)
    sum_ndcg=sum_ndcg+x
    # print(x)
plain_ndcg=sum_ndcg/reco_matrix[o].shape[0]

plain_ndcg
# 0.06877010356676778

0.03442249577000286

In [ ]:
0.04792797790372785
0.04599395586314128
0.04278003454306751
0.03944891646901801
0.036196512457131354
0.03410950136728672
0.03422949673067553
0.034110177325415714
0.03442249577000286